In [ ]:
hyper_params = {
    "input_size": 224,
    "test_size": 0.1,
    "batch_size": 32,
    "num_epochs": 50,
    "learning_rate": 2e-4,
    "early_stop": 5,
    "betas": (0.9, 0.999), 
    "weight_decay": 0.01
}

# Import Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!pip install jcopdl

In [ ]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config
import cv2

import PIL
from PIL import Image

import os

# device = torch.device('cuda:0')
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
device

# Dataset and Dataloader

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [ ]:
class BoneAgeDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = None):
        self.df = pd.read_csv(csv_path)
        self.df.male = self.df.male.astype(int)
        self.images_folder = images_folder
        self.transform = transform
        self.mean_bone_age = self.df['boneage'].mean()
        self.std_bone_age = self.df['boneage'].std()

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        filename = str(self.df.iloc[index].id)+".png"
        label = torch.from_numpy(np.asarray([self.df.iloc[index].boneage]))
        label_z = torch.from_numpy(np.asarray([(self.df.iloc[index].boneage-self.mean_bone_age)/self.std_bone_age]))
        gender = torch.from_numpy(np.asarray([self.df.iloc[index].male]))
        image = PIL.Image.open(os.path.join(self.images_folder, filename)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image.float(), label_z.float(), gender.float()

In [ ]:
transform = transforms.Compose([
    transforms.Resize((hyper_params["input_size"], hyper_params["input_size"])),
    transforms.RandomRotation(degrees=20),
    transforms.RandomAffine(degrees=20, scale=(0.9, 0.9), translate=(0.2, 0.2)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

In [ ]:
data_train = BoneAgeDataset("../bone-age/boneage-training-dataset/boneage-training-dataset.csv", "../Segmentation/final-dataset/", transform)

# Split into Train Val
random_seed = 21
torch.manual_seed(random_seed)
test_size = int(len(data_train) * hyper_params["test_size"])
train_size = len(data_train) - (test_size)

train_set, test_set = random_split(
    data_train,
    [train_size, test_size]
)

trainloader = DataLoader(train_set, batch_size=hyper_params['batch_size'], shuffle=True)
testloader = DataLoader(test_set, batch_size=hyper_params['batch_size'])

In [ ]:
len(train_set), len(test_set)

# Architecture and Config

In [ ]:
import torch
from torch import nn
import timm

class BoneAgeModel(nn.Module):
    def __init__(self, pretrained=True):
        super(BoneAgeModel, self).__init__()
        self.backbone = timm.create_model('tf_efficientnet_b7', pretrained=pretrained, in_chans=3)
        self.n_features = self.backbone.classifier.in_features
        self.backbone.reset_classifier(0)
        self.fc1 = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.n_features, 256),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256+1, 1)
        )
        
#         for param in self.backbone.clf.parameters():
#             param.requires_grad = True
        
#         for param in self.backbone.blocks[6].parameters():
#             param.requires_grad = True
        
        
    def freeze(self):
        for param in self.backbone.parameters():
            param.requires_grad = False # Freezing Weight
        for param in self.fc.parameters():
            param.requires_grad = False
    
    def unfreeze(self):
        for param in self.backbone.parameters():
            param.requires_grad = True # Unfreezing Weight
        for param in self.fc.parameters():
            param.requires_grad = True

    def forward(self, images, genders):
        features = self.backbone(images)
        # features = (bs, embedding_size)
        output1 = self.fc1(features)
        x = torch.cat([output1, genders], dim=1)
        output2 = self.fc2(x)
        # outputs  = (bs, num_classes)
        return output2
    
model = BoneAgeModel(pretrained=True)
model.to(device)
# model.unfreeze()

# Training

In [ ]:
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import mean_squared_error
# from torch.nn import HuberLoss

In [ ]:
# def RMSE(outputs, targets):
#     return torch.sqrt(nn.MSELoss()(outputs.view(-1), targets.view(-1)))
def RMSE(outputs, targets):
    return torch.sqrt(nn.MSELoss()(outputs.view(-1)*data_train.std_bone_age+data_train.mean_bone_age, targets.view(-1)*data_train.std_bone_age+data_train.mean_bone_age))
#     return HuberLoss()
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=hyper_params['learning_rate'], betas=hyper_params['betas'], weight_decay=hyper_params['weight_decay'])
scheduler = StepLR(optimizer, step_size=25, gamma=0.5)

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
#     scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    running_rmse = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, targets, genders) in bar:         
        images = images.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        genders = genders.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
#         with amp.autocast(enabled=True):
        outputs = model(images, genders)
        loss = criterion(outputs, targets)
        loss = loss / 1
        rmse = RMSE(outputs, targets)
            
        loss.backward()
        optimizer.step()
        
            
#         scaler.scale(loss).backward()
    
# #         if (step + 1) % 1 == 0:
#         scaler.step(optimizer)
#         scaler.update()

        # zero the parameter gradients
        optimizer.zero_grad()
                
        running_loss += (loss.item() * batch_size)
        running_rmse += (rmse.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_rmse = running_rmse / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
        
        
    if scheduler is not None:
        scheduler.step()

    gc.collect()
    
    return epoch_loss, epoch_rmse

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [ ]:
@torch.no_grad()
def test_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    running_rmse = 0.0
    
    TARGETS = []
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, targets, genders) in bar:        
        images = images.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        genders = genders.to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        outputs = model(images, genders)
        loss = criterion(outputs, targets)
        rmse = RMSE(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        running_rmse += (rmse.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_rmse = running_rmse / dataset_size
        
        PREDS.append(outputs.view(-1).cpu().detach().numpy())
        TARGETS.append(targets.view(-1).cpu().detach().numpy())
        
        bar.set_postfix(Epoch=epoch, Test_Loss=epoch_loss, Test_RMSE = epoch_rmse,
                        LR=optimizer.param_groups[0]['lr'])   
    
    TARGETS = np.concatenate(TARGETS)*data_train.std_bone_age+data_train.mean_bone_age
    PREDS = np.concatenate(PREDS)*data_train.std_bone_age+data_train.mean_bone_age
    
#     outputs.view(-1)*data_train.std_bone_age+data_train.mean_bone_age
#     targets.view(-1)*data_train.std_bone_age+data_train.mean_bone_age
    
    test_rmse = mean_squared_error(TARGETS, PREDS, squared=False)
    gc.collect()
    
    test_mae = np.mean(np.abs(TARGETS - PREDS))
    
    test_smape = np.mean(2.0 * np.abs(TARGETS - PREDS) / ((np.abs(TARGETS) + np.abs(PREDS)) +1e-10 ))
    
    return epoch_loss, test_rmse, test_mae, test_smape

In [ ]:
import time
import copy
from collections import defaultdict
import gc
from torch.cuda import amp
from tqdm import tqdm

In [ ]:
from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

In [ ]:
!mkdir Model-EfficientNet-Full-0.3-v2

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_rmse = np.inf
    history = defaultdict(list)
    df = pd.DataFrame.from_dict(history)
    early_stop = 0
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss, train_epoch_rmse = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=trainloader, 
                                           device=device, epoch=epoch)
        
        test_epoch_loss, test_epoch_rmse, test_epoch_mae, test_epoch_smape = test_one_epoch(model, testloader, 
                                                         device=device, 
                                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Test Loss'].append(test_epoch_loss)
        history['Test RMSE'].append(test_epoch_rmse)
        history['Test MAE'].append(test_epoch_mae)
        history['Test SMAPE'].append(test_epoch_smape)
        
        df = pd.DataFrame.from_dict(history)
        df.to_csv('EfficientNet-CLAHE-0.3-v2.csv', index=False)
        
        print(f'Test RMSE: {test_epoch_rmse}')
        print(f'Test MAE: {test_epoch_mae}')
        
        if test_epoch_rmse <= best_epoch_rmse:
            print(f"{c_}Test Loss Improved ({best_epoch_rmse} ---> {test_epoch_rmse})")
            best_epoch_rmse = test_epoch_rmse
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop = 1
            
        else:
            early_stop += 1
#             print(f"Early stop {early_stop}: Epoch {epoch}")
#             if(early_stop == hyper_params['early_stop']):
#                 break
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best RMSE: {:.4f}".format(best_epoch_rmse))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    PATH = "Model-EfficientNet-Full-0.3-v2/RMSE{:.4f}_epoch{:.0f}.bin".format(best_epoch_rmse, epoch)
    torch.save(model.state_dict(), PATH)
    
    
    return model, history

In [ ]:
model, history = run_training(model, optimizer, scheduler,
                              device=device,
                              num_epochs=50)

Sebelumnya: RMSE: 11.8938, MAE: 9.2201

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].plot(history['Train Loss'])
axs[0, 0].title.set_text('Train RMSE')
axs[0, 1].plot(history['Valid RMSE'])
axs[0, 1].title.set_text('Valid RMSE')
axs[1, 0].plot(history['Valid MAE'])
axs[1, 0].title.set_text('Valid MAE')
axs[1, 1].plot(history['Valid SMAPE'])
axs[1, 1].title.set_text('Valid SMAPE')

In [ ]:
test_epoch_loss, test_epoch_rmse, test_epoch_mae, test_epoch_smape = test_one_epoch(model, testloader, 
                                                         device=device, 
                                                         epoch=1)

In [ ]:
test_epoch_rmse

In [ ]:
0.2603*data_train.std_bone_age+data_train.mean_bone_age